This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [3]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 


Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [8]:
import requests
import collections
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the "V3" quandl api at `https://www.quandl.com/api/v3/`

In [5]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
sample = requests.get(f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key={API_KEY}')

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
sample.text

'{"dataset_data":{"limit":1,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2000-06-07","end_date":"2020-12-01","frequency":"daily","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null]],"collapse":null,"order":null}}'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

I. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017.

In [9]:
# Use API call to get data
# Data is kinda long when I use requests.get(url) and then request.json(url)
# Better to divide into sections
start = '2017-01-01'
end = '2017-12-31'
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
api_call = f'{url}?start_date={start}&end_date={end}&api_key={API_KEY}'

In [10]:
# Get the JSON data using the requests package
r = requests.get(api_call)
r

<Response [200]>

II. Convert the returned JSON object into a Python dictionary.

In [14]:
ds = r.json()
type(ds)

dict

In [16]:
# Checking keys for the dictionary
ds.keys()

dict_keys(['dataset_data'])

In [18]:
# Let's find the call for column names
ds['dataset_data'].keys()

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])

In [19]:
ds['dataset_data']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

III. Calculate what the highest and lowest opening prices were for the stock in this period.

In [21]:
open_index= {}

for col in ds['dataset_data']['data']:
    if col[1] is not None:
        open_index[col[0]] = col[1]
        
max_open_value = max(open_index.values())
max_date = {v: k for k, v in open_index.items()}[max_open_value]
min_open_value = min(open_index.values())
min_date = {v: k for k, v in open_index.items()}[min_open_value]
print('Highest opening price value was: ' + str(max_open_value) + ' on ' + str(max_date)) 
print('Lowest opening price value was: ' + str(min_open_value) + ' on ' + str(min_date))

Highest opening price value was: 53.11 on 2017-12-14
Lowest opening price value was: 34.0 on 2017-01-24


IV. What was the largest change in any one day (based on High and Low price)?

In [24]:
change = {}

for col in ds['dataset_data']['data']:
    if col[2] is not None and col[3] is not None:
        change[col[0]] = col[2] - col[3]
        
max_oneday_price_change = max(change.values())
max_oneday_change_date = {v: k for k, v in change.items()}[max_oneday_price_change]

print('Maximum single day price change was: ' + str(round(max_oneday_price_change,2)) + ' on ' + str(max_oneday_change_date))

Maximum single day price change was: 2.81 on 2017-05-11


V. What was the largest change between any two days (based on Closing Price)?

In [25]:
change_two_d = []
yesterday = 0

for daily in ds['dataset_data']['data']:
    if yesterday == 0:
        yesterday = daily[4]
    else:
        change_two_d.append(daily[4]-yesterday)
        yesterday = daily[4]
        
max_change = max(change_two_d)
min_change = min(change_two_d)

if abs(min_change) > max_change:
    print('Maximum change between any two days based on Closing Price is -$%.2f.' % abs(min_change))
else:
    print('Maximum change between any two days based on Closing Price is $%.2f.' % max_change)

Maximum change between any two days based on Closing Price is $2.56.


VI. What was the average daily trading volume during this year?

In [29]:
trade_vol = 0

for daily in ds['dataset_data']['data']:
    trade_vol += daily[6]
    average = trade_vol / len(ds['dataset_data']['data'])
print('Average daily trading volume during 2017 is', average)

Average daily trading volume during 2017 is 89124.33725490196


VII. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [31]:
trade_vol = sorted([daily[6] for daily in ds['dataset_data']['data']])
trade_len = len(trade_vol)
half = trade_len/2
if trade_len % 2 == 1:
    
    print("Median trading volume during 2017 is %d." % trade_vol[round(half)])

Median trading volume during 2017 is 76600.
